### heisernberg1D with 2 spin per site

In [131]:
from __future__ import print_function, division
import sys,os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='10' # set number of MKL threads to run in parallel
#
quspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,quspin_path)
from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator
from quspin.tools.lanczos import lanczos_full,lanczos_iter,FTLM_static_iteration,LTLM_static_iteration
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
from scipy import sparse
import scipy

In [119]:
I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

Sz1 = sparse.kron(I,Sz,format='csr')
Sz2 = sparse.kron(Sz,I,format='csr')
SzSz = sparse.kron(Sz,Sz,format='csr')

Sx1 = sparse.kron(I,Sx,format='csr')
Sx2 = sparse.kron(Sx,I,format='csr')
SxSx = sparse.kron(Sx,Sx,format='csr')

Sy1 = sparse.kron(I,Sy,format='csr')
Sy2 = sparse.kron(Sy,I,format='csr')
SySy = sparse.kron(Sy,Sy,format='csr')


h1 = -(sparse.kron(Sz1, Sz2,format='csr') 
       - sparse.kron(Sx1, Sx2,format='csr')
       - sparse.kron(Sy1, Sy2,format='csr') 
     ).real

h2 = -(sparse.kron(SzSz, np.identity(4),format='csr') 
       - sparse.kron(SxSx, np.identity(4),format='csr')
       - sparse.kron(SxSx, np.identity(4),format='csr') 
     ).real * (1/2)

h3 = -(sparse.kron(np.identity(4),SzSz, format='csr') 
       - sparse.kron(np.identity(4),SxSx, format='csr')
       - sparse.kron(np.identity(4), SySy, format='csr') 
     ).real * (1/2)

lh = h1 + h2 + h3

In [120]:
def make_local_ham(pos, hams, sps = 2, L = 10):
    assert(len(pos) == len(hams), "size of position and hams must be the same")
    assert(sps == hams[0].shape[0], "dimension of hamiltonian is inconsistent")
    
    cnt = 0
    h = sparse.identity(1, dtype=np.float64)
    for i in range(L):
        tmp = sparse.identity(sps)
        if i in pos:
#             print(f"insert operator at {i}th site")
            tmp = hams[cnt]
            cnt+=1
        h = sparse.kron(tmp, h, format='csr')
    return h

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-120-26772de5078c>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(pos) == len(hams), "size of position and hams must be the same")
<ipython-input-120-26772de5078c>:3: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(sps == hams[0].shape[0], "dimension of hamiltonian is inconsistent")


In [187]:
L = 8
L_ = int(L/2)


H = sparse.csr_matrix((4**L_, 4**L_), dtype=np.float64)
for i in range(L_):
    tmp = make_local_ham([2*i + 1, (2*(i+1))%L], [Sz,Sz], 2, L)
    tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sx,Sx], 2, L)
    tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sy,Sy], 2, L) 
    
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sz,Sz/2], 2, L)
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sx,Sx/2], 2, L)
    tmp += make_local_ham([2*i, (2*i+1)%L], [Sy,Sy/2], 2, L) 
    
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sz,Sz/2], 2, L)
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sx,Sx/2], 2, L)
    tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sy,Sy/2], 2, L)
    H += tmp

In [188]:
E_, _ = scipy.linalg.eig(H.toarray())
E_.sort()

### construct with quspin

In [189]:
basis = spin_basis_1d(L,pauli=False)
J_zz = [[1,i,(i+1)%L] for i in range(L)] # OBC
J_xy = [[1/2.0,i,(i+1)%L] for i in range(L)] # OBC
ops_dict = dict(Jpm=[["+-",J_xy]],Jmp=[["-+",J_xy]],Jzz=[["zz",J_zz]])
H2 = quantum_operator(ops_dict,basis=basis,dtype=np.float64, check_symm=False)

Hermiticity check passed!


In [190]:
E = H2.eigvalsh({})
np.linalg.norm(E - E_)

7.65494925668007e-14

In [183]:
E[:10]

array([-3.65109341, -3.12841906, -3.12841906, -3.12841906, -2.69962815,
       -2.45873851, -2.45873851, -2.45873851, -2.45873851, -2.45873851])

In [184]:
H

array([-3.65109341, -3.12841906, -3.12841906, -3.12841906, -2.69962815,
       -2.45873851, -2.45873851, -2.45873851, -2.45873851, -2.45873851])

### construct local hamiltonian for worm algorithm

In [192]:
L = 4
tmp = make_local_ham([2*i + 1, (2*(i+1))%L], [Sz,Sz], 2, L)
tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sx,Sx], 2, L)
tmp += make_local_ham([2*i + 1, (2*(i+1))%L], [Sy,Sy], 2, L) 

tmp += make_local_ham([2*i, (2*i+1)%L], [Sz,Sz/2], 2, L)
tmp += make_local_ham([2*i, (2*i+1)%L], [Sx,Sx/2], 2, L)
tmp += make_local_ham([2*i, (2*i+1)%L], [Sy,Sy/2], 2, L) 

tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sz,Sz/2], 2, L)
tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sx,Sx/2], 2, L)
tmp += make_local_ham([(2*i+2)%L, (2*i+3)%L], [Sy,Sy/2], 2, L)
np.save("array/test_model.npy", tmp.real.toarray())

In [196]:
print(tmp.diagonal())

[ 1.125+0.j -0.125+0.j  0.875+0.j  0.125+0.j  1.125+0.j -0.125+0.j
  0.875+0.j  0.125+0.j  0.125+0.j -1.125+0.j -0.125+0.j -0.875+0.j
  0.125+0.j -1.125+0.j -0.125+0.j -0.875+0.j]
